## Optimizing Retail Banking: Data Ingestion, Cleaning, and EDA

This notebook ingests the provided banking transactions dataset, performs systematic data cleaning, and conducts exploratory data analysis (EDA) following best practices. The goal is to prepare high-quality data and surface insights that will support RFM-based customer segmentation in subsequent steps.

### Objectives
- Load dataset and validate schema against project instructions
- Clean types, missing values, duplicates, and inconsistent categories
- Assess outliers and data quality
- Explore univariate and bivariate distributions
- Summarize customer behavior and transaction patterns
- Prepare and export cleaned data and intermediate artifacts

> Dataset path: `data/bank_data_C.csv`


In [ ]:
# Imports and configuration
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Plotting aesthetics
sns.set(style="whitegrid", context="notebook")
plt.rcParams.update({
    "figure.figsize": (10, 6),
    "axes.titlesize": 14,
    "axes.labelsize": 12,
})

# Paths
PROJECT_ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
RAW_PATH = DATA_DIR / "bank_data_C.csv"
PROCESSED_DIR = DATA_DIR / "processed"
PROCESSED_DIR.mkdir(exist_ok=True, parents=True)

RAW_PATH


### Data dictionary (from instructions)
- TransactionID: Unique transaction ID
- CustomerID: Unique customer ID
- CustomerDOB: Date of birth
- CustGender: Customer gender
- CustLocation: Customer location
- CustAccountBalance: Current account balance
- TransactionDate: Date of transaction
- TransactionTime: Timestamp of transaction
- TransactionAmount: Value of transaction

We'll validate these fields after loading the CSV.


In [ ]:
# Load dataset
assert RAW_PATH.exists(), f"Expected CSV not found at {RAW_PATH}"

df = pd.read_csv(RAW_PATH)
print(df.shape)
df.head(3)


In [ ]:
# Initial inspection
info_buf = []
df_info = df.dtypes.astype(str).rename("dtype").to_frame()
df_info["non_null"] = df.notnull().sum()
df_info["null"] = df.isnull().sum()
df_info["unique"] = df.nunique()
df_info



### Data cleaning plan
We'll apply the following steps:
1. Fix data types for dates/times and numeric fields
2. Standardize categorical values (e.g., gender casing)
3. Remove exact duplicate rows, then deduplicate per `TransactionID`
4. Handle missing values with context-aware strategies
5. Identify and treat outliers in `TransactionAmount` and `CustAccountBalance`
6. Validate referential logic (e.g., non-negative amounts, reasonable DOB)
7. Create clean dataset snapshot


We parsed date/time columns first so later temporal analyses work reliably. Any unparsable values are set to missing (`NaT`).


In [ ]:
# Numeric conversions
for col in ["CustAccountBalance", "TransactionAmount"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
"Numeric conversions complete"


We converted monetary fields to numeric, coercing invalid strings to `NaN`. This ensures stats/plots work without errors.


In [ ]:
# Categorical standardization
if "CustGender" in df.columns:
    df["CustGender"] = (
        df["CustGender"].astype(str).str.strip().str.title().replace({"Nan": np.nan})
    )

if "CustLocation" in df.columns:
    df["CustLocation"] = df["CustLocation"].astype(str).str.strip().replace({"nan": np.nan, "": np.nan})
"Categorical cleaning complete"


Gender and location values are standardized to consistent casing and trimmed. Obvious placeholders like empty strings are treated as missing.


In [ ]:
# Parse date and time columns
for col in ["CustomerDOB", "TransactionDate"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# Parse TransactionTime if present
if "TransactionTime" in df.columns and not np.issubdtype(df["TransactionTime"].dtype, np.datetime64):
    try:
        df["TransactionTime"] = pd.to_datetime(df["TransactionTime"], errors="coerce")
    except Exception:
        pass

"Date/time parsing complete"


In [ ]:
# Deduplication
before = len(df)
# Remove exact duplicate rows
df = df.drop_duplicates()

# Ensure unique TransactionID if present
if "TransactionID" in df.columns:
    dup_tx = df["TransactionID"].duplicated(keep=False)
    print({"duplicate_transaction_ids": int(dup_tx.sum())})
    df = df.drop_duplicates(subset=["TransactionID"], keep="first")

after = len(df)
{"removed_duplicates": before - after, "rows_remaining": after}


In [ ]:
# Missing values overview
missing = df.isnull().mean().sort_values(ascending=False)
missing = missing[missing > 0]
missing


### Missing value handling strategy
- `CustomerDOB`: leave missing if unparseable; optional imputation later for modeling
- `CustGender`, `CustLocation`: leave missing; consider 'Unknown' label if needed in dashboards
- `CustAccountBalance`, `TransactionAmount`: numeric imputation not advised without domain context; leave missing or drop only when required for specific analyses
- `TransactionDate`/`Time`: records without a valid date are not usable for temporal analyses and will be dropped


In [ ]:
# Apply minimal, conservative row filtering for critical fields
rows_before = len(df)

# Drop rows with no valid TransactionDate
df = df[~df["TransactionDate"].isna()] if "TransactionDate" in df.columns else df

# Ensure TransactionAmount exists for transaction-level EDA
df = df[~df["TransactionAmount"].isna()] if "TransactionAmount" in df.columns else df

rows_after = len(df)
{"dropped_rows": rows_before - rows_after, "rows_remaining": rows_after}


In [ ]:
# Outlier detection (univariate)
outlier_summary = {}
for col in ["TransactionAmount", "CustAccountBalance"]:
    if col in df.columns:
        series = df[col].dropna()
        if series.empty:
            continue
        q1, q3 = series.quantile([0.25, 0.75])
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        outlier_summary[col] = {
            "iqr_lower": float(lower),
            "iqr_upper": float(upper),
            "num_lower": int((series < lower).sum()),
            "num_upper": int((series > upper).sum()),
        }
outlier_summary


### Outlier treatment approach
We'll apply winsorization caps for visualization stability but retain raw values in a separate copy for reference. This balances robustness in charts with data fidelity for modeling.


In [ ]:
# Schema overview (concise)
df.info()


 next step 

In [ ]:
# Create capped copy for EDA visuals
eda = df.copy()
for col in ["TransactionAmount", "CustAccountBalance"]:
    if col in eda.columns:
        lower, upper = eda[col].quantile([0.01, 0.99])
        eda[col] = eda[col].clip(lower=lower, upper=upper)

"Capped columns for EDA visuals"


## Exploratory Data Analysis (EDA)


In [ ]:
# Univariate distributions (numeric)
num_cols = [c for c in ["TransactionAmount", "CustAccountBalance"] if c in eda.columns]
for col in num_cols:
    sns.histplot(eda[col].dropna(), kde=True)
    plt.title(f"Distribution of {col}")
    plt.show()


In [ ]:
# Univariate distributions (categorical)
cat_cols = [c for c in ["CustGender", "CustLocation"] if c in eda.columns]
for col in cat_cols:
    order = eda[col].value_counts(dropna=False).index
    sns.countplot(y=col, data=eda, order=order)
    plt.title(f"Counts of {col}")
    plt.show()


In [ ]:
# Temporal patterns if TransactionDate present
if "TransactionDate" in eda.columns:
    eda["TransactionDate"] = pd.to_datetime(eda["TransactionDate"])  # idempotent
    eda["YearMonth"] = eda["TransactionDate"].dt.to_period("M").astype(str)
    monthly = eda.groupby("YearMonth")["TransactionAmount"].agg(["count", "sum"]).reset_index()
    fig, ax1 = plt.subplots()
    sns.lineplot(data=monthly, x="YearMonth", y="count", ax=ax1, label="Transactions Count")
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha="right")
    ax1.legend(loc="upper left")
    ax2 = ax1.twinx()
    sns.lineplot(data=monthly, x="YearMonth", y="sum", ax=ax2, color="orange", label="Total Amount")
    ax2.legend(loc="upper right")
    plt.title("Monthly Transactions and Amount")
    plt.tight_layout()
    plt.show()


In [ ]:
# Customer-level summaries
if "CustomerID" in eda.columns:
    customer_tx = (
        eda.groupby("CustomerID").agg(
            num_transactions=("TransactionID", "nunique") if "TransactionID" in eda.columns else ("TransactionAmount", "count"),
            total_amount=("TransactionAmount", "sum"),
            avg_amount=("TransactionAmount", "mean"),
            last_tx_date=("TransactionDate", "max") if "TransactionDate" in eda.columns else ("TransactionAmount", "idxmax"),
        )
    )
    customer_tx = customer_tx.sort_values("total_amount", ascending=False)
    customer_tx.head(10)
